##  augmentation of grouped PNGs in folder, save origins and results in one new folder

In [5]:
import os
import csv
from PIL import Image, ImageEnhance
import numpy as np
import random
import shutil

# Define augmentation functions
def flip_image(img, horizontal=True):
    "Flip the image horizontally or vertically."
    return img.transpose(Image.FLIP_LEFT_RIGHT) if horizontal else img.transpose(Image.FLIP_TOP_BOTTOM)

def zoom_image(img, zoom_factor=1.2):
    "Zoom into the image by a specified factor."
    width, height = img.size
    x = int(width / zoom_factor)
    y = int(height / zoom_factor)
    img_cropped = img.crop(((width - x) // 2, (height - y) // 2, (width + x) // 2, (height + y) // 2))
    return img_cropped.resize((width, height), Image.LANCZOS)

def adjust_brightness(img, factor=1.5):
    "Adjust the brightness of the image."
    enhancer = ImageEnhance.Brightness(img)
    return enhancer.enhance(factor)

def add_noise(img, noise_level=0.1):
    "Add random noise to the image."
    img_array = np.array(img)
    noise = np.random.normal(0, noise_level * 255, img_array.shape).astype(np.uint8)
    img_noisy = np.clip(img_array + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(img_noisy)

# Function to randomly apply one augmentation
def random_augmentation(img):
    augmentations = [flip_image, zoom_image, adjust_brightness, add_noise]
    augmentation = random.choice(augmentations)
    return augmentation(img)

# Read CSV and get streak labels
def read_csv(csv_path):
    streak_labels = {}
    rows = []
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            streak_labels[row['output']] = int(row['label'])
            rows.append(row)
    return streak_labels, rows

# Save the updated CSV
def save_csv(output_csv_path, old_rows, new_rows, fieldnames):
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(old_rows)  # Write original CSV content
        writer.writerows(new_rows)  # Add new rows for augmentations

# Main processing function
def process_images(input_folder, csv_path):
    streak_labels, old_rows = read_csv(csv_path)

    output_folder = os.path.join(input_folder, "augmented")
    os.makedirs(output_folder, exist_ok=True)

    # Prepare the output CSV
    csv_name = os.path.basename(csv_path)
    output_csv_path = os.path.join(output_folder, csv_name)
    fieldnames = ['input', 'output', 'label']
    new_rows = []

    # Copy all original files to the output folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith('.png'):
            file_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Copy original file
            shutil.copy(file_path, output_path)

            # Augment images if streaks are present
            if streak_labels.get(filename, 0) == 1:
                for iteration in range(1, 3):  # Two augmentations per image
                    img = Image.open(file_path)
                    img_augmented = random_augmentation(img)

                    # Save augmented image
                    augmented_filename = f"{os.path.splitext(filename)[0]}_{iteration}.png"
                    augmented_output_path = os.path.join(output_folder, augmented_filename)
                    img_augmented.save(augmented_output_path)

                    # Add augmented file info to new rows
                    new_rows.append({'input': filename, 'output': augmented_filename, 'label': 1})

    # Save the updated CSV with old and new rows
    save_csv(output_csv_path, old_rows, new_rows, fieldnames)
    print(f"Processing complete. Outputs saved in '{output_folder}'.")

# Set the path to your folder containing PNG images and CSV file
input_folder = './Data/fits_filtered4'  # Replace with the path to your folder
csv_path = os.path.join(input_folder, 'dictionary_0.csv')  # Replace with the name of your CSV file
process_images(input_folder, csv_path)


Processing complete. Outputs saved in './Data/fits_filtered4\augmented'.
